In [1]:
from google.colab import drive
drive.mount('/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /gdrive


In [2]:
cd ..

/


In [3]:
cd 'gdrive/My Drive/gan'

/gdrive/My Drive/gan


# **Finding Path Names**

In [0]:
# import os
# root =os.walk('data/img_align_celeba_/')
# total=list(root)
# paths=[]
# for hint in total[1:]:
#     path,_,files = hint
#     temp=[path+'/'+x for x in files]
#     paths.append(temp)
# import pandas as pd
# path_flatten=[actual  for x in paths for actual in x]
# # path_flatten
# file_names=pd.DataFrame(path_flatten,columns=['name'])
# file_names['id']=pd.Series([x[3] for x in file_names['name'].str.split('/')])

# file_names=file_names[file_names['id'].str.len()>4]

# file_names['name'] = file_names['name'].apply(lambda x : x[5:])
# file_names['id']=file_names['id'].apply(lambda x : int(x.split('.')[0]))
# file_names = file_names.set_index('id')
# file_names.sort_index(inplace=True)
# file_names.to_csv('file_names',index=True)

# Reading filenames

In [0]:
import pandas as pd

In [0]:
file_names=pd.read_csv('file_names')

# Importing Tensorflow and other necesarry Modules

In [5]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import re
import numpy as np
import scipy.misc

import tensorflow as tf
import tensorflow.contrib.slim as slim

import my_time

from Tflib import session

import data
import models

import datetime
from functools import partial
import traceback
import os


The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



# Imlib

In [0]:



def immerge(images, row, col):
    """Merge images into an image with (row * h) * (col * w).

    `images` is in shape of N * H * W(* C=1 or 3)
    """
    h, w = images.shape[1], images.shape[2]
    if images.ndim == 4:
        img = np.zeros((h * row, w * col, images.shape[3]))
    elif images.ndim == 3:
        img = np.zeros((h * row, w * col))
    for idx, image in enumerate(images):
        i = idx % col
        j = idx // col
        img[j * h:j * h + h, i * w:i * w + w, ...] = image

    return img



In [0]:

def to_range(images, min_value=0.0, max_value=1.0, dtype=None):
    """Transform images from [-1.0, 1.0] to [min_value, max_value] of dtype."""
    assert np.min(images) >= -1.0 - 1e-5 and np.max(images) <= 1.0 + 1e-5 \
        and (images.dtype == np.float32 or images.dtype == np.float64), \
        ('The input images should be float64(32) '
         'and in the range of [-1.0, 1.0]!')
    if dtype is None:
        dtype = images.dtype
    return ((images + 1.) / 2. * (max_value - min_value) +
            min_value).astype(dtype)


In [0]:
def imwrite(image, path):
    """Save an [-1.0, 1.0] image."""
    if image.ndim == 3 and image.shape[2] == 1:  # for gray image
        image = np.array(image, copy=True)
        image.shape = image.shape[0:2]
    return scipy.misc.imsave(path, to_range(image, 0, 255, np.uint8))


# Pylib

In [0]:

def mkdir(paths):
    
    if not isinstance(paths, (list, tuple)):
        paths = [paths]
    for path in paths:
        if not os.path.isdir(path):
            os.makedirs(path)


# Tflib

In [0]:
def load_checkpoint(ckpt_dir_or_file, session, var_list=None):

    if os.path.isdir(ckpt_dir_or_file):
        ckpt_dir_or_file = tf.train.latest_checkpoint(ckpt_dir_or_file)

    restorer = tf.train.Saver(var_list)
    restorer.restore(session, ckpt_dir_or_file)
    print(' [*] Loading checkpoint succeeds! Copy variables from % s!' % ckpt_dir_or_file)

In [0]:

def summary(tensor_collection,
            summary_type=['mean', 'stddev', 'max', 'min', 'sparsity', 'histogram'],
            scope=None):
    """Summary.

    usage:
        1. summary(tensor)
        2. summary([tensor_a, tensor_b])
        3. summary({tensor_a: 'a', tensor_b: 'b})
    """
    def _summary(tensor, name, summary_type):
        """Attach a lot of summaries to a Tensor."""
        if name is None:
            # Remove 'tower_[0-9]/' from the name in case this is a multi-GPU training
            # session. This helps the clarity of presentation on tensorboard.
            name = re.sub('%s_[0-9]*/' % 'tower', '', tensor.name)
            name = re.sub(':', '-', name)

        summaries = []
        if len(tensor.shape) == 0:
            summaries.append(tf.summary.scalar(name, tensor))
        else:
            if 'mean' in summary_type:
                mean = tf.reduce_mean(tensor)
                summaries.append(tf.summary.scalar(name + '/mean', mean))
            if 'stddev' in summary_type:
                mean = tf.reduce_mean(tensor)
                stddev = tf.sqrt(tf.reduce_mean(tf.square(tensor - mean)))
                summaries.append(tf.summary.scalar(name + '/stddev', stddev))
            if 'max' in summary_type:
                summaries.append(tf.summary.scalar(name + '/max', tf.reduce_max(tensor)))
            if 'min' in summary_type:
                summaries.append(tf.summary.scalar(name + '/min', tf.reduce_min(tensor)))
            if 'sparsity' in summary_type:
                summaries.append(tf.summary.scalar(name + '/sparsity', tf.nn.zero_fraction(tensor)))
            if 'histogram' in summary_type:
                summaries.append(tf.summary.histogram(name, tensor))
        return tf.summary.merge(summaries)

    if not isinstance(tensor_collection, (list, tuple, dict)):
        tensor_collection = [tensor_collection]

    with tf.name_scope(scope, 'summary'):
        summaries = []
        if isinstance(tensor_collection, (list, tuple)):
            for tensor in tensor_collection:
                summaries.append(_summary(tensor, None, summary_type))
        else:
            for tensor, name in tensor_collection.items():
                summaries.append(_summary(tensor, name, summary_type))
        return tf.summary.merge(summaries)


In [0]:

def counter(start=0, scope=None):
    with tf.variable_scope(scope, 'counter'):
        counter = tf.get_variable(name='counter',
                                  initializer=tf.constant_initializer(start),
                                  shape=(),
                                  dtype=tf.int64)
        update_cnt = tf.assign(counter, tf.add(counter, 1))
        return counter, update_cnt

In [0]:

def tensors_filter(tensors, filters, combine_type='or'):
    assert isinstance(tensors, (list, tuple)), '`tensors` shoule be a list or tuple!'
    assert isinstance(filters, (str, list, tuple)), '`filters` should be a string or a list(tuple) of strings!'
    assert combine_type == 'or' or combine_type == 'and', "`combine_type` should be 'or' or 'and'!"

    if isinstance(filters, str):
        filters = [filters]

    f_tens = []
    for ten in tensors:
        if combine_type == 'or':
            for filt in filters:
                if filt in ten.name:
                    f_tens.append(ten)
                    break
        elif combine_type == 'and':
            all_pass = True
            for filt in filters:
                if filt not in ten.name:
                    all_pass = False
                    break
            if all_pass:
                f_tens.append(ten)
    return f_tens

In [0]:
def trainable_variables(filters=None, combine_type='or'):
    t_var = tf.trainable_variables()
    if filters is None:
        return t_var
    else:
        return tensors_filter(t_var, filters, combine_type)

In [0]:

# ==============================================================================
# =                                    param                                   =
# ==============================================================================
def init_settings():
    
    global experiment_name,atts,n_att,img_size,shortcut_layers,inject_layers,enc_dim,dec_dim,dis_dim,dis_fc_dim,enc_layers,dec_layers,dis_layers,\
     mode,epoch,batch_size,lr_base,n_d,b_distribution,thres_int,test_int,n_sample,use_cropped_img,experiment_name
    atts =  ['Bald', 'Bangs', 'Black_Hair', 'Blond_Hair', 'Brown_Hair', 'Bushy_Eyebrows', 'Eyeglasses', 'Male', 'Mouth_Slightly_Open', 'Mustache', 'No_Beard', 'Pale_Skin', 'Young']
    n_att = len(atts)
    img_size = 128
    shortcut_layers = 1
    inject_layers = 1
    enc_dim = 64
    dec_dim = 64
    dis_dim = 64
    dis_fc_dim = 1024
    enc_layers = 5
    dec_layers = 5
    dis_layers = 5
    # training
    mode = 'wgan'
    epoch = 200
    batch_size = 32
    lr_base = 0.0002
    n_d = 5
    b_distribution = 'none'
    thres_int = 0.5
    test_int = 1
    n_sample = 64
    # others
    use_cropped_img = False
    experiment_name = 'training'
    
    
def intiliaze_models():
    # models
    Genc = partial(models.Genc, dim=enc_dim, n_layers=enc_layers)
    Gdec = partial(models.Gdec, dim=dec_dim, n_layers=dec_layers, shortcut_layers=shortcut_layers, inject_layers=inject_layers)
    D = partial(models.D, n_att=n_att, dim=dis_dim, fc_dim=dis_fc_dim, n_layers=dis_layers)

    return Genc,Gdec,D



In [0]:

def load_ckpt():
    ckpt_dir = 'output/%s/'%experiment_name
    

    try:
        load_checkpoint(ckpt_dir, sess)
    except:
        sess.run(tf.global_variables_initializer())
    return ckpt_dir

In [0]:

# data
def load_train_data():
    

    tr_data = data.Celeba(file_names.name.tolist(),'data/', atts, img_size, batch_size, part='train', sess=sess, crop=not use_cropped_img)
    val_data = data.Celeba(file_names.name.tolist(),'data/', atts, img_size, n_sample, part='val', shuffle=False, sess=sess, crop=not use_cropped_img)
    return tr_data,val_data


In [18]:

sess = session()


In [19]:
sess.list_devices()

[_DeviceAttributes(/job:localhost/replica:0/task:0/device:CPU:0, CPU, 268435456, 7568465941638759721),
 _DeviceAttributes(/job:localhost/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 17179869184, 13607084344740819421),
 _DeviceAttributes(/job:localhost/replica:0/task:0/device:XLA_GPU:0, XLA_GPU, 17179869184, 4243338834018270873),
 _DeviceAttributes(/job:localhost/replica:0/task:0/device:GPU:0, GPU, 7470045594, 110126357559606888)]

In [21]:

init_settings()

Genc,Gdec,D = intiliaze_models()


tr_data,val_data = load_train_data()



lr = tf.placeholder(dtype=tf.float32, shape=[])

xa = tr_data.batch_op[0]
a = tr_data.batch_op[1]
b = tf.random_shuffle(a)
_a = (tf.to_float(a) * 2 - 1) * thres_int
if b_distribution == 'none':
    _b = (tf.to_float(b) * 2 - 1) * thres_int
elif b_distribution == 'uniform':
    _b = (tf.to_float(b) * 2 - 1) * tf.random_uniform(tf.shape(b)) * (2 * thres_int)
elif b_distribution == 'truncated_normal':
    _b = (tf.to_float(b) * 2 - 1) * (tf.truncated_normal(tf.shape(b)) + 2) / 4.0 * (2 * thres_int)

xa_sample = tf.placeholder(tf.float32, shape=[None, img_size, img_size, 3])
_b_sample = tf.placeholder(tf.float32, shape=[None, n_att])

# generate
z = Genc(xa)
xb_ = Gdec(z, _b)
with tf.control_dependencies([xb_]):
    xa_ = Gdec(z, _a)

# discriminate
xa_logit_gan, xa_logit_att = D(xa)
xb__logit_gan, xb__logit_att = D(xb_)

# discriminator losses
if mode == 'wgan':  # wgan-gp
    wd = tf.reduce_mean(xa_logit_gan) - tf.reduce_mean(xb__logit_gan)
    d_loss_gan = -wd
    gp = models.gradient_penalty(D, xa, xb_)
elif mode == 'lsgan':  # lsgan-gp
    xa_gan_loss = tf.losses.mean_squared_error(tf.ones_like(xa_logit_gan), xa_logit_gan)
    xb__gan_loss = tf.losses.mean_squared_error(tf.zeros_like(xb__logit_gan), xb__logit_gan)
    d_loss_gan = xa_gan_loss + xb__gan_loss
    gp = models.gradient_penalty(D, xa)
elif mode == 'dcgan':  # dcgan-gp
    xa_gan_loss = tf.losses.sigmoid_cross_entropy(tf.ones_like(xa_logit_gan), xa_logit_gan)
    xb__gan_loss = tf.losses.sigmoid_cross_entropy(tf.zeros_like(xb__logit_gan), xb__logit_gan)
    d_loss_gan = xa_gan_loss + xb__gan_loss
    gp = models.gradient_penalty(D, xa)

xa_loss_att = tf.losses.sigmoid_cross_entropy(a, xa_logit_att)

d_loss = d_loss_gan + gp * 10.0 + xa_loss_att

# generator losses
if mode == 'wgan':
    xb__loss_gan = -tf.reduce_mean(xb__logit_gan)
elif mode == 'lsgan':
    xb__loss_gan = tf.losses.mean_squared_error(tf.ones_like(xb__logit_gan), xb__logit_gan)
elif mode == 'dcgan':
    xb__loss_gan = tf.losses.sigmoid_cross_entropy(tf.ones_like(xb__logit_gan), xb__logit_gan)

xb__loss_att = tf.losses.sigmoid_cross_entropy(b, xb__logit_att)
xa__loss_rec = tf.losses.absolute_difference(xa, xa_)

g_loss = xb__loss_gan + xb__loss_att * 10.0 + xa__loss_rec * 100.0

#global_step
global_step = tf.Variable(initial_value=0,
                          name='global_step', trainable=False)

# optim
d_var = trainable_variables('D')
d_step = tf.train.AdamOptimizer(lr, beta1=0.5).minimize(d_loss, var_list=d_var)

g_var = trainable_variables('G')
g_step = tf.train.AdamOptimizer(lr, beta1=0.5).minimize(g_loss, var_list=g_var,global_step=global_step)

# summary
d_summary = summary({
    d_loss_gan: 'd_loss_gan',
    gp: 'gp',
    xa_loss_att: 'xa_loss_att',
}, scope='D')

g_summary = summary({
    xb__loss_gan: 'xb__loss_gan',
    xb__loss_att: 'xb__loss_att',
    xa__loss_rec: 'xa__loss_rec',
}, scope='G')

lr_summary = summary({lr: 'lr'}, scope='Learning_Rate')

d_summary = tf.summary.merge([d_summary, lr_summary])

# sample
x_sample = Gdec(Genc(xa_sample, is_training=False), _b_sample, is_training=False)


# ==============================================================================
# =                                    train                                   =
# ==============================================================================

# iteration counter
it_cnt, update_cnt = counter()

# saver
saver = tf.train.Saver(max_to_keep=1)

# summary writer
summary_writer = tf.summary.FileWriter('output/summaries', sess.graph)

# initialization

ckpt_dir = load_ckpt()


 # train
try:
    # data for sampling
    xa_sample_ipt, a_sample_ipt = val_data.get_next()
    b_sample_ipt_list = [a_sample_ipt]  # the first is for reconstruction
    for i in range(len(atts)):
        tmp = np.array(a_sample_ipt, copy=True)
        tmp[:, i] = 1 - tmp[:, i]   # inverse attribute
        tmp = data.Celeba.check_attribute_conflict(tmp, atts[i], atts)
        b_sample_ipt_list.append(tmp)

    it_per_epoch = len(tr_data) // (batch_size * (n_d + 1))
    max_it = epoch * it_per_epoch
    for it in range(sess.run(it_cnt), max_it):
        with my_time.Timer(is_output=False) as t:
            sess.run(update_cnt)

            # which epoch
            epoch = it//it_per_epoch
            it_in_epoch = it%it_per_epoch + 1

            # learning rate
            lr_ipt = lr_base / (10 ** (epoch // 100))

            # train D
            with tf.device('/device:XLA_GPU:0'):
                
                for i in range(n_d):
                    d_summary_opt, _ = sess.run([d_summary, d_step], feed_dict={lr: lr_ipt})
                summary_writer.add_summary(d_summary_opt, it)
            with tf.device('/device:GPU:0'):
                # train G
                g_summary_opt, _ = sess.run([g_summary, g_step], feed_dict={lr: lr_ipt})
                summary_writer.add_summary(g_summary_opt, it)

            i_global = sess.run(global_step)

            # display
            if (it + 1) % 1 == 0:
                print("Epoch: (%3d) (%5d/%5d) Time: %s! ,global_step :(%d)" % (epoch, it_in_epoch, it_per_epoch, t,i_global))




            # save
            if (it + 1) % 700 == 0:
                save_path = saver.save(sess, '%s/Epoch_(%d)_(%dof%d).ckpt' % (ckpt_dir, epoch, it_in_epoch, it_per_epoch),global_step=global_step)
                print('Model is saved at %s!' % save_path)


        # sample
            if (it + 1) % 100 == 0:
                x_sample_opt_list = [xa_sample_ipt, np.full((n_sample, img_size, img_size // 10, 3), -1.0)]
                for i, b_sample_ipt in enumerate(b_sample_ipt_list):
                    _b_sample_ipt = (b_sample_ipt * 2 - 1) * thres_int
                    if i > 0:   # i == 0 is for reconstruction
                        _b_sample_ipt[..., i - 1] = _b_sample_ipt[..., i - 1] * test_int / thres_int
                    x_sample_opt_list.append(sess.run(x_sample, feed_dict={xa_sample: xa_sample_ipt, _b_sample: _b_sample_ipt}))
                sample = np.concatenate(x_sample_opt_list, 2)

                save_dir = 'output/%s/sample_training' % experiment_name
                mkdir(save_dir)
                imwrite(immerge(sample, n_sample, 1), '%s/Epoch_(%d)_(%dof%d)(g:%d).jpg' % (save_dir, epoch, it_in_epoch, it_per_epoch,i_global))
except:
    traceback.print_exc()
finally:
    save_path = saver.save(sess, '%s/Epoch_(%d)_(%dof%d).ckpt' % (ckpt_dir, epoch, it_in_epoch, it_per_epoch),global_step=global_step)
    print('Model is saved at %s!' % save_path)
    sess.close()

NameError: ignored

# Testing

In [0]:
sess=session()

In [23]:

init_settings()
Genc,Gdec,D = intiliaze_models()

te_data = data.Celeba('data', atts, img_size, 1, part='test', sess=sess, crop=not use_cropped_img)

xa_sample = tf.placeholder(tf.float32, shape=[None, img_size, img_size, 3])
_b_sample = tf.placeholder(tf.float32, shape=[None, n_att])

x_sample = Gdec(Genc(xa_sample, is_training=False), _b_sample, is_training=False)

load_ckpt()
 #sample
try:
    for idx, batch in enumerate(te_data):
        xa_sample_ipt = batch[0]
        a_sample_ipt = batch[1]
        b_sample_ipt_list = [a_sample_ipt]  # the first is for reconstruction
        for i in range(len(atts)):
            tmp = np.array(a_sample_ipt, copy=True)
            tmp[:, i] = 1 - tmp[:, i]   # inverse attribute
            tmp = data.Celeba.check_attribute_conflict(tmp, atts[i], atts)
            b_sample_ipt_list.append(tmp)

        x_sample_opt_list = [xa_sample_ipt, np.full((1, img_size, img_size // 10, 3), -1.0)]
        for i, b_sample_ipt in enumerate(b_sample_ipt_list):
            _b_sample_ipt = (b_sample_ipt * 2 - 1) * thres_int
            if i > 0:   # i == 0 is for reconstruction
                _b_sample_ipt[..., i - 1] = _b_sample_ipt[..., i - 1] * test_int / thres_int
            x_sample_opt_list.append(sess.run(x_sample, feed_dict={xa_sample: xa_sample_ipt, _b_sample: _b_sample_ipt}))
        sample = np.concatenate(x_sample_opt_list, 2)

        save_dir = 'output/sample_testing'
            
        if not os.path.exists(save_dir):
            mkdir(save_dir)
        
        imwrite(sample.squeeze(0), '%s/%d.png' % (save_dir, idx + 182638))

        print('%d.png done!' % (idx + 182638))

except:
    traceback.print_exc()
finally:
    sess.close()

Exception ignored in: <bound method Dataset.__del__ of <data.Celeba object at 0x7f887e8acbe0>>
Traceback (most recent call last):
  File "/gdrive/My Drive/gan/data.py", line 81, in __del__
    if self._sess:
AttributeError: 'Celeba' object has no attribute '_sess'


TypeError: ignored

# New Section